# Conversion Tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="blog-2019-12-converting-whole-slide-images.jpg" style="height:300px" />



The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.

In [1]:
%%time
!bioformats2raw --help

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--version] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-directory=<memoDirectory>]
                    [--pyramid-name=<pyramidName>] [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-st

## Required OME-Zarr options
Two of the options are currently necessary to produce OME-Zarr data:
```
      --file_type=<fileType>
                     Tile file extension: n5, zarr (default: n5) [Can break
                       compatibility with raw2ometiff]
```
and
```
     --dimension-order=<dimensionOrder>
                     Override the input file dimension order in the output file
                       [Can break compatibility with raw2ometiff] (XYZCT,
                       XYZTC, XYCTZ, XYCZT, XYTCZ, XYTZC)
```

`--file_type` which produces Zarr output rather than N5 as the intermediate format. If we additionally pass the `--dimension-order` argument, then the intermediate result can be used directly by the ome-zarr library.

In [2]:
%%time
!bioformats2raw i2k2020.gif $PWD --file_type=zarr --dimension-order=XYZCT --scale-format-string=gif.zarr/%d

2020-12-01 12:25:46,540 [main] INFO  c.g.bioformats2raw.Converter - Output will be incompatible with raw2ometiff (pyramidName: data.zarr, scaleFormatString: gif.zarr/%d)
2020-12-01 12:25:47,075 [main] INFO  loci.formats.ImageReader - GIFReader initializing i2k2020.gif
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Verifying GIF format
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Reading dimensions
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Reading data blocks
2020-12-01 12:25:47,082 [main] INFO  loci.formats.FormatHandler - Populating metadata
2020-12-01 12:25:47,295 [main] INFO  c.g.bioformats2raw.Converter - Using 1 pyramid resolutions
2020-12-01 12:25:47,295 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 128 (tileWidth: 1024) sizeY 128 (tileWidth: 1024) imageCount 8
2020-12-01 12:25:47,559 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 128
2020-12-01 12:25:47,559 [m

In [3]:
!ls *.ome.xml data.zarr

METADATA.ome.xml

data.zarr:
gif.zarr


## Moving to the cloud

You can then move the generated output to S3

In [4]:
!mc cp --recursive data.zarr/gif.zarr/ play/i2k2020/gif.zarr/

...5.0.0.0.0:  4.19 KiB / 4.19 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  5.10 KiB/s 0s

You can see your image under http://hms-dbmi.github.io/vizarr?source=https://play.minio.io:9000/i2k2020/gif.zarr

In [5]:
!mc cat play/i2k2020/gif.zarr/.zattrs

{"multiscales":[{"datasets":[{"path":"0"}],"version":"0.1"}]}

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.